# Coordinate Calculator and Label Compiler

Inputs:
- location in pixels of beginning and end of axes
- range of axes
- units of axis range
- user's desired spacing for x coordinates
- NO legend input! how to specify different colors/markers that separates the curves (for later stages)
    - could eventually have mask look for the legend (distinctive enough) to learn what types of distinguishing features exist between the lines on the graph
    - I will ignore this part for now
- detectron2 output:
    - list of dicts, one dict for each image
    - each dict has an object 'instances' with the following fields.
        - pred_masks: a tensor of shape (N,H,W) for N instances and an image of height H and width W.
        - access it using: ```outputs['instances'].pred_masks```
    
Outputs:
- dictionary with the following fields:
    - coordinates: a dictionary, with fields equal to the curve labels (one for each label), and entries that are a list of tuples, each with (x, and y coordinates in same units as input image)
    - units: (a tuple of x and y units)
    - beginning and end of curve
    - NO labels (for later stages): a list of labels for each curve, taken from legend if possible, or automatic (data1, data2, etc)


Options for coordinates:
1. based on the actual axis, choose a set of x coordinates for which we want to know the y coordinates (for each curve)
2. based on pixels, grab points that occur every "few" pixels along the x axis, then convert to coordinates

- could do the first option by finding out how many pixels are between each of the given x points, and using that number of pixels paired with the second option.
- do we want user to input the spacing between points, or should we use a default value that seems reasonable based on the range of the x axis?
- should the different curves have uniform x axis values (so y values are directly comparable), or should we have the sets of data points for each curve start at the 'beginning' of the curve and end at the 'end'


Outlier cases:
- nested plots
- weird legend formatting

uniform x axis but also save the very first and last endpoints of the curve 

maybe incorporate later:

convert ID to the label from the legend input

def get_label_for_id():
    return label

### Notes, TODO
- adjust the x and y pixel locations to lie on the same horizontal and vertical lines
- put the coordinate and pixel min max values in a dictionary or something more compact
- use os to create a directory for each project. currently puts excel files in the current folder

In [1]:
import pandas as pd
import numpy as np

This is the first function called in my code. The arguments are all of the user input that I need

In [5]:
# create the axis info dictionary
def get_axis_info(xcoordinatemin, xcoordinatemax, xpixelmin, xpixelmax, ycoordinatemin, ycoordinatemax, ypixelmin, ypixelmax, max_points, units):
    # if the input from kev is in a more compact form extract this info below before passing into the scale funcitons
    x_scale = get_x_scale(xcoordinatemin, xcoordinatemax, xpixelmin, xpixelmax)
    y_scale = get_y_scale(ycoordinatemin, ycoordinatemax, ypixelmin, ypixelmax)
    pixel_origin = (xpixelmin, ypixelmax) # assumes that y pixel max is the larger y value
    axis_info_dict = {'pixel_origin': pixel_origin,
                      'x_scale': x_scale,
                      'y_scale': y_scale,
                      'step': get_step(max_points, xpixelmin, xpixelmax),
                      'units': units,
                      'y_pixel_range': (ypixelmin, ypixelmax),
                      'x_pixel_range': (xpixelmin, xpixelmax)}
    return axis_info_dict

In [ ]:
def test_get_axis_info():
    # the output is a dictionary with the fields: pixel_origin, x_scale, y_scale, step, and units
    return

In [6]:
# convert the max points desired into a step size in pixels
def get_step(max_points, xpixelmin, xpixelmax):
    #step = length of x axis in pixels / max_points
    step = (xpixelmax - xpixelmin) / max_points
    return step

In [ ]:
def test_get_step():
    # the step size * the number of points should be close to the length of the axis
    # step size is an integer
    # the length of the axis/ step size should be close to but less than the max points
    return

In [7]:
# establish scaling from pixel to real units
def get_x_scale(xcoordinatemin, xcoordinatemax, xpixelmin, xpixelmax):
    # the x pixel and x coordinate count up in the same direction
    pixel_range = xpixelmax - xpixelmin
    coordinate_range = xcoordinatemax - xcoordinatemin
    x_scale = pixel_range / coordinate_range
    return x_scale # pixels per coordinate


def get_y_scale(ycoordinatemin, ycoordinatemax, ypixelmin, ypixelmax):
    # y pixel count down and y coordinate count up from origin
    pixel_range = ypixelmin - ypixelmax
    coordinate_range = ycoordinatemax - ycoordinatemin
    y_scale = pixel_range / coordinate_range
    return y_scale # pixels per coordinate

In [ ]:
def test_get_x_scale():
    # x_scale should be greater than 1 (or maybe not depending on the units)
    # x_scale * coordinate range should equal pixel range
    return

In [ ]:
def test_get_y_scale():
    # y_scale should be greater than zero
    # y_scale * coordinate range should equal pixel range
    return

In [8]:
# convert pixel location to coordinates
def pixel_to_coords(pixel_loc, axis_info_dict):
    # pixel_loc is a tuple (x,y) of pixel location starting from top left
    x_pixel_loc = pixel_loc[0]
    coord_x = x_pixel_to_coords(x_pixel_loc, axis_info_dict)
    
    # get signed distance from pixel to origin in x and y(pixel units):
    pixel_distance_y = axis_info_dict['pixel_origin'][1] - pixel_loc[1]
    
    # pixels / (pixel/coord) = coord
    coord_y = pixel_distance_y / axis_info_dict['y_scale']
    return (coord_x, coord_y)

In [ ]:
def test_pixel_to_coords():
    # given a scale and a location, test a few cases (+-0)
    # the output coordinates are within the coordinate ranges for each axis
    return

In [9]:
# for use later in code, just the x axis pixel to coords
def x_pixel_to_coords(x_pixel_loc, axis_info_dict):
    pixel_distance_x = x_pixel_loc - axis_info_dict['pixel_origin'][0]
    coord_x = pixel_distance_x / axis_info_dict['x_scale']
    return coord_x

In [11]:
# dont really need this. accomplished in the unify x function
# pretty much get the avg y pixel function for each unique x pixel value
def clean_pixel_lst(pixel_lst):
    return cleaned_pixel_lst

In [12]:
# find closest number in list to value val
def closest(lst, val):
    lst = np.asarray(lst)
    idx = (np.abs(lst - val)).argmin()
    return lst[idx]

In [ ]:
def test_closest():
    # val is equidistant to two values in list
    # val is equal to one value in list
    # val is closer to one in particular
    return

In [13]:
# return x and y values of the unified list
def unify_x(pixel_lst, axis_info_dict):
    # get the number of pixels between each desired coordinate pt based on scale
    # create bins of pixels and avg the values between them
    pixel_lst.sort()
    
    # step is a global variable based on user input
    step = axis_info_dict['step']
    x_end = len(pixel_lst) / step
    x_vals = [range(0, x_end, step)]
    
    # create dictionary of point and closest standard x val
    closest_dict = {}
    for point in pixel_lst:
        key = closest(x_vals, point[0])
        if key in closest_dict:
            closest_dict[key].append(point)
        else:
            closest_dict[key] = [point]
    
    # iterate through keys to average all y values in each set
    for key in closest_dict:
        y_vals = [i[1] for i in closest_dict[key]]
        y_val = sum(y_vals) / len(y_vals)
        closest_dict[key] = y_val
    
    # for all the missing dict keys, make a line between nearest values and fill it in
    existing_keys = list(closest_dict.keys())
    existing_keys.sort()
    for x in x_vals:
        if x not in existing_keys:
            # find the index of first existing x greater than x
            i = 0
            while existing_keys[i] < x and i < (len(x_vals + 1)):
                i += 1
            
            x2 = existing_keys[i] # existing x just above missing x
            y2 = closest_dict[x2]
            x1 = existing_keys[i-1] # existing x just below missing x
            y1 = closest_dict[x1]
            
            # find line between bounds
            m = (y1 - y2) / (x1 - x2)
            b = (x1 * y2 - x2 * y1) / (x1 - x2)
            
            # solve for y of x
            y = m * x + b
            closest_dict[x] = y

        else:
            continue
    
    # turn dictionary into a list of tuples
    unified_pixel_lst = list(closest_dict.items())
    unified_pixel_lst.sort()
    
    
    return unified_pixel_lst # a list of (x, y) tuples

In [ ]:
def test_unify_x():
    # the x values in the list of tuples are all stepsize apart
    # the x values are unique
    # y values are all between the min and max pixel
    # same for x values
    return

## modify to use the instances mask which gives true false for each pixel for each instance

note: tensor size (num instances, y, x)

detectron2 output:
- list of dicts, one dict for each image
    - each dict has an object 'instances' with the following fields.
        - pred_masks: a tensor of shape (N,H,W) for N instances and an image of height H and width W.
        - access it using: ```outputs['instances'].pred_masks``` in the case of one image

In [10]:
# modified


# looks at one N and all H,W values in that 'slice'
# each curve is one ID. get all pixel locations for one ID
def get_pixels_2d(pixel_array_2d):
    result = np.where(pixel_array == True)
    pixel_lst = list(zip(result[1], result[0])) # gives the x then the y in the tuple
    return pixel_lst

In [ ]:
def test_get_pixels_2d():
    # create an array with a few trues and many falses, check positions in output match
    return

In [12]:
zeros = np.random.rand(2,3,4)
zeros

array([[[0.84455676, 0.89897016, 0.78674855, 0.69721505],
        [0.49250018, 0.95144026, 0.56515559, 0.07534797],
        [0.67122673, 0.17205716, 0.7801582 , 0.91178312]],

       [[0.54522823, 0.08518038, 0.15126383, 0.96199722],
        [0.05317068, 0.80030109, 0.81247336, 0.0199464 ],
        [0.60822706, 0.43958936, 0.01509645, 0.01112308]]])

In [21]:
loc = np.where(zeros > 0.6)
loc

(array([0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1]),
 array([0, 0, 0, 0, 1, 2, 2, 2, 0, 1, 1, 2]),
 array([0, 1, 2, 3, 1, 0, 2, 3, 3, 1, 2, 0]))

In [14]:
# modified

# ID = curve_N
# step 1 to create coordinate dictionary to add to the output dict
def create_pixel_dict(pred_masks):
    # initialize dict
    pixel_dict = {}
    
    pixel_array_3d = np.array(pred_masks)
    for N in range(len(pixel_array_3d)):
        pixel_array_2d = pixel_array_3d[N]
        pixel_lst = get_pixels_2d(pixel_array_2d)
        
        # add the list of pixels for this N to the pixel dict
        ID = 'curve_' + str(N)
        pixel_dict[ID] = pixel_lst
    return pixel_dict

In [ ]:
def test_create_pixel_dict():
    # check that the keys are 'curve_' and then a unique number that matches the shape of pred_masks
    # the values are not the same for each one (not repeated)
    return

In [15]:
# create coordinate dictionary to add to the output dict
def create_coordinate_dict(pixel_dict, axis_info_dict):
    # initialize dict
    coordinate_dict = {}
    
    for ID in pixel_dict.keys():
        pixel_lst = pixel_dict[ID]
        
        # get unified x axis:
        # add an if statement to handle user specifiying either step size (in coordinates) or number of points
        unified_pixel_lst = unify_x(pixel_lst, axis_info_dict)
        
        # convert pixels to coordinates
        coordinate_lst = []
        for pixel_loc in unified_pixel_lst:
            coordinate_lst.append(pixel_to_coords(pixel_loc, axis_info_dict))
        
        # add the list of coordinates for this ID to the coordinate dict
        coordinate_dict[str(ID)] = coordinate_lst
    return coordinate_dict

In [ ]:
def test_create_coordinate_dict():
    # the keys of the coordinate dict are the same as that of the pixel dict
    return

In [16]:
# get the coordinate locations of the start and end of the curve
def get_start_end(pixel_dict):
    # initialize dict
    start_end_dict = {}
    
    for ID in pixel_dict.keys():
        pixel_lst = pixel_dict[ID]
        
        # get start and end, assumes x is in the first position
        start = x_pixel_to_coords(min(pixel_lst)[0])
        end = x_pixel_to_coords(max(pixel_lst)[0])
        # add the start and end tuple for this ID to the pixel dict
        start_end_dict[str(ID)] = (start, end)
    return start_end_dict

In [ ]:
def test_get_start_end():
    # the keys of the output dict are the same as the input
    # the end is larger than the start
    # the start and end are between the coordinate ranges
    return

In [17]:
# modified



# create the output dictionary and input the units of axis and labels directly from the input info,
# as well as the coordinate dictionary
def create_output_dict(pred_masks, axis_info_dict):
    pixel_dict = create_pixel_dict(pred_masks)
    output_dict = {}
    output_dict['coordinates'] = create_coordinate_dict(pixel_dict, axis_info_dict)
    output_dict['start_end'] = get_start_end(pixel_dict)
    output_dict['units'] = axis_info_dict['units']
    return output_dict

In [19]:
# saves the resulting coordinates and start/stop info in an excel file with one sheet per curve
def write_results_to_excel(output_dict, filename):
    excel_filename = str(filename) + '.xlsx'
    writer = pd.ExcelWriter(excel_filename, engine='xlsxwriter')
    x_units = output_dict['units'][0]
    y_units = output_dict['units'][1]
    
    # a summary of the start and end of the x axis for each ID
    starts = []
    ends = []
    ids = []
    for ID in output_dict[start_end].keys():
        ids.append(ID)
        start = output_dict['start_end'][ID][0]
        end = output_dict['start_end'][ID][1]
        starts.append(start)
        ends.append(end)
    df = pd.DataFrame([ids, starts, ends],  columns=['ID', 'x start '+str(x_units), 'x end '+str(x_units)])
    df.to_excel(writer, sheet_name='starts_ends')
    
    # the actual data in xy form, one ID per sheet
    for ID in output_dict['coordinates'].keys():
        x = output_dict['coordinates'][ID][0]
        y = output_dict['coordinates'][ID][1]
        column_titles = ['x, ' + str(x_units), 'y, ' + str(y_units)]
        df = pd.DataFrame([x, y], columns=column_titles)
        df.to_excel(writer, sheet_name=str(ID))
    writer.save()

In [20]:
# modified - might need to check if there are more than one image before indexing for multiple


# loops through each image in the output of detectron2
def datayoink_to_csv(detect_output, axis_info_dict):
    n_images = len(detect_output)
    for image in range(n_images):
        pred_masks = detect_output[image]['instances'].pred_masks
        output_dict = create_output_dict(pred_masks, axis_info_dict)
        filename = 'image_' + str(image)
        write_results_to_excel(output_dict, filename)
    return

In order to use these functions, code looks like this:

In [21]:
# detect_output is the output of detectron2 - a list of dictionaries
axis_info_dict = get_axis_info(xcoordinatemin, xcoordinatemax, xpixelmin, xpixelmax, ycoordinatemin, ycoordinatemax, ypixelmin, ypixelmax, max_points, units)
datayoink_to_csv(detect_output, axis_info_dict)

NameError: name 'xcoordinatemin' is not defined